In [245]:
import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.feature_selection import RFECV

from factor_analyzer import FactorAnalyzer
from factor_analyzer import calculate_kmo
from factor_analyzer import calculate_bartlett_sphericity

from joblib import dump, load

In [ ]:
#############################################################################################################################################

In [165]:
# Original large_cat_desc_col_names.csv: 1903 rows; columns 1..3973 + ddG
lg_df = pd.read_csv('data/large_cat_desc_col_names.csv')
lg_df_cleaned = lg_df[lg_df['ddG'] != 0]
lg_df_cleaned # 1903 x 3975. 

,Identifier,1,2,3,4,5,6,7,8,9,...,3965,3966,3967,3968,3969,3970,3971,3972,3973,ddG
0,1_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194
1,1_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.880495,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684
2,1_1_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898,0.276786
3,1_1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194
4,1_1_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633,1.463011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,9_9_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.880495,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684
1899,9_9_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.104378,1.789966,1.707972,-0.078629,-1.416062,-1.347984,-1.383415,0.492188,0.691898,0.276786
1900,9_9_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194
1901,9_9_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.112732,0.601039,0.691527,1.360684,0.614598,0.524781,0.570340,-1.518452,-0.230633,1.463011


In [166]:
label_df = lg_df_cleaned['Identifier'] # 0..1902 in order
y = lg_df_cleaned['ddG'] 
X = lg_df_cleaned.drop(lg_df_cleaned.columns[0], axis=1) 
X = X.drop(X.columns[3973], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 1332; 591. 

# Need to extract the row indices corresponding to the training & test split at this stage: 
idx_train = X_train.index.values.tolist()
idx_test = X_test.index.values.tolist() 
label_train_df = label_df[idx_train] 
label_test_df = label_df[idx_test]

In [252]:
# OLS: create, fit, evaluate on test-set predictions. 
ols = LinearRegression()
ols.fit(X_train, y_train)
ols_predictions = ols.predict(X_test)

In [168]:
# PLS: create, fit, evaluate on test-set predictions.
pls = PLSRegression(n_components=2)
pls.fit(X_train, y_train)
pls_predictions = pls.predict(X_test)

def pls_predict_ee(properties):
    return pls.predict(properties.reshape(1,-1)) 

PLS mean squared error:  0.53568590422573
PLS mean absolute error:  0.5548903409235344


In [170]:
# Applying variance filter solely based on X_train. 
thresh = VarianceThreshold(0.03) # tuned. 
X_train_VarReduced = thresh.fit_transform(X_train) # (1332, 252)
indices_VarReduced = thresh.get_support(indices=True) # list: retained column indices

(1332, 252)

In [173]:
# Applying correlation filter solely based on X_train. 
df = pd.DataFrame(X_train_VarReduced)
corr_matrix = df.corr().abs() 
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) # top half of triangular correlation matrix.
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)] # prepare to drop indices with high correlation
df.drop(to_drop, axis=1, inplace=True) # drop said indices
X_train_FullyReduced = df.to_numpy() # ndarray: (1332, 171)

# list: retained column indices
indices_FullyReduced = []
for i in range(len(indices_VarReduced)):
    if i in to_drop: pass 
    else: indices_FullyReduced.append(indices_VarReduced[i])

(1332, 171)

In [230]:
# Now we need to get only the features from X_test which correspond to our feature reduction above. 
# Need to use the list comprehension below to avoid an off-by-one error!
X_test_FullyReduced_df = X_test.iloc[:,[i-1 for i in indices_FullyReduced]]
X_test_FullyReduced = X_test_FullyReduced_df.to_numpy() # (571, 171)

In [234]:
# KMO for factor analysis suitability: measures the proportion of variance that could be common variance. 
kmo_all, kmo_model = calculate_kmo(X_train_FullyReduced)
print('KMO: ', kmo_model)

KMO:  0.8695679817445267


/Users/davidpolefrone/anaconda3/lib/python3.11/site-packages/factor_analyzer/utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


In [ ]:
#############################################################################################################################################

In [235]:
# Exploratory factor analysis to determine how many factors we end up keeping in a second pass. 
factor_model = FactorAnalyzer(n_factors=len(X_train_FullyReduced),rotation="varimax")
factor_model.fit(X_train_FullyReduced) 
eigenvalues, _ = factor_model.get_eigenvalues()
number_of_factors = sum(eigenvalues > 1) 
number_of_factors

18

In [236]:
### Second-pass factor analysis on the training set: 171 features -> 18 factors.
factor_model = FactorAnalyzer(n_factors=number_of_factors, rotation="varimax")
factor_model.fit(X_train_FullyReduced)

FactorAnalyzer(n_factors=18, rotation='varimax', rotation_kwargs={})

In [240]:
# Transforming into factor space, then rescaling. 
X_train_faLV = factor_model.transform(X_train_FullyReduced) # (1332, 18)
X_test_faLV = factor_model.transform(X_test_FullyReduced) # (571, 18)

scaler = StandardScaler()
X_train_faLV_scaled = scaler.fit_transform(X_train_faLV)  # (1332, 18)
X_test_faLV_scaled = scaler.fit_transform(X_test_faLV)  # (571, 18)

In [254]:
# CV scores for FA-derived models. 
k=4	# Four-fold CV default across all models. 
print(f'k-fold score using OLSReg w/ FA LVs = {sum(cross_val_score(LinearRegression(), X_train_faLV_scaled, y_train, cv=k))/k:.4f}')
print(f'k-fold score using Random Forest w/ FA LVs = {sum(cross_val_score(RandomForestRegressor(random_state=42), X_train_faLV_scaled, y_train, cv=k))/k:.4f}')
print(f'k-fold score using LassoCV w/ FA LVs = {sum(cross_val_score(LassoCV(), X_train_faLV_scaled, y_train, cv=k))/k:.4f}')
print(f'k-fold score using RidgeCV w/ FA LVs = {sum(cross_val_score(RidgeCV(), X_train_faLV_scaled, y_train, cv=k))/k:.4f}')
print(f'k-fold score using ElasticNetCV w/ FA LVs = {sum(cross_val_score(ElasticNetCV(), X_train_faLV_scaled, y_train, cv=k))/k:.4f}')

k-fold score using OLSReg w/ FA LVs = 0.6224
k-fold score using Random Forest w/ FA LVs = 0.8704
k-fold score using LassoCV w/ FA LVs = 0.6270
k-fold score using RidgeCV w/ FA LVs = 0.6224
k-fold score using ElasticNetCV w/ FA LVs = 0.6242
k-fold score using PLSReg w/ FA LVs = 0.6224


In [243]:
# CV scores for PLS and PLS-RF. Likely overcorrelated...
print(f'k-fold score using PLSReg (implicit LVs) = {sum(cross_val_score(PLSRegression(n_components=number_of_factors), X_train_FullyReduced, y_train, cv=k))/k:.4f}')
pls = PLSRegression(n_components=number_of_factors).fit(X_train_FullyReduced, y_train) 
X_train_plsLV = pls.transform(X_train_FullyReduced)
X_test_plsLV = pls.transform(X_test_FullyReduced)
print(f'k-fold score using Random Forest w/ PLS18 LVs = {sum(cross_val_score(RandomForestRegressor(random_state=42), X_train_plsLV, y_train, cv=k))/k:.4f}') 

k-fold score using PLSReg (implicit LVs) = 0.9998
k-fold score using Random Forest w/ PLS16 LVs = 0.9896


In [246]:
# RFE'ing factors: 18 factors -> 3 remaining factors. 
rfe = LinearRegression()
rfe = RFECV(rfe, step=1, cv=5)
rfe = rfe.fit(X_train_faLV_scaled, y_train)
X_train_faLV_RFE = rfe.transform(X_train_faLV_scaled) # 3 factors remain. 
X_test_faLV_RFE = rfe.transform(X_test_faLV_scaled) 

In [250]:
# Select CV scores for FA-RFE-derived models. 
print(f'k-fold score using OLSReg w/ FA-RFE = {sum(cross_val_score(LinearRegression(), X_train_faLV_RFE, y_train, cv=k))/k:.4f}')
print(f'k-fold score using RF w/ FA-RFE = {sum(cross_val_score(RandomForestRegressor(random_state=42), X_train_faLV_RFE, y_train, cv=k))/k:.4f}')
print(f'k-fold score using LassoCV w/ FA-RFE = {sum(cross_val_score(LassoCV(), X_train_faLV_RFE, y_train, cv=k))/k:.4f}')

k-fold score using OLSReg w/ FA-RFE = 0.6341
k-fold score using RF w/ FA-RFE = 0.9193
k-fold score using LassoCV w/ FA-RFE = 0.6341


In [ ]:
#############################################################################################################################################
#############################################################################################################################################
#############################################################################################################################################

In [140]:
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, iterations=100, cd_iterations=10, step_size=0.01):
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations): # 100 steps
        for i in range(len(optimized_catalysts)):
            original_ee = pls_predict_ee(optimized_catalysts[i])  # Call the loaded-in regression model
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations): # "coordinate descent iterations"
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
    return optimized_catalysts